#  ATOC5860 Application Lab #3 - eigenfaces
##### Written by Dr. Vineel Yettella (ATOC Ph.D. 2018, now at Apple), with additional commenting from Prof. Kay (CU)
##### last updated February 22, 2023 - works with environment atoc5860env2023clean.yml

### LEARNING GOALS:
1) Complete an EOF analysis using Singular Value Decomposition (SVD). 
2) Provide a qualitative description of the results. What are the eigenvalues, the eigenvectors, and the principal components?  What do you learn from each one about the space-time structure of your underlying dataset?
3) Provide a qualitative description of the results: What are the eigenvalue,the eigenvector, and the principal component?  What do they mean physically? What do you learn from each one about the space-time structure of the underlying dataset?
4) Reconstruct original data using a subset of EOFs: How many EOFs are needed to reconstruct faces?

### DATA and UNDERLYING SCIENCE MOTIVATION:  
In this notebook, you apply EOF analysis to a standard database for facial recognition: the At&t database. 

*“Our Database of Faces, (formerly 'The ORL Database of Faces'), contains a set of face images taken between April 1992 and April 1994 at the lab. The database was used in the context of a face recognition project carried out in collaboration with the Speech, Vision and Robotics Group of the Cambridge University Engineering Department.

There are ten different images of each of 40 distinct subjects. For some subjects, the images were taken at different times, varying the lighting, facial expressions (open / closed eyes, smiling / not smiling) and facial details (glasses / no glasses). All the images were taken against a dark homogeneous background with the subjects in an upright, frontal position (with tolerance for some side movement).”*

The goal is to think a bit “out of the box” of Atmospheric and Oceanic Sciences about potential applications for the methods you are learning in this class.  And also to think about some of the pitfalls of using statistics for facial recognition. Are the faces here representative of all faces?

### Non-exhaustive Questions to guide your analysis:  
1) Execute all code without making any modifications. What do the EOFs (spatial patterns) tell you?  What do the PCs tell you?  How do you interpret what you are finding?
    - EOFs are	the	orthogonal	spatial	patterns	which	explain	the	temporal variance	in	our	dataset.	In	this	example,	EOFs	represent	the	unique	facial	features	which	explain	the	difference	between	faces.	Principal	components	tell	us	the	amplitude	of	each	EOF	across	the	sample	dimension.	So	in	our	case	EOFs	tell	us	the	unique	facial	features	and	the	PCs	tell	us	 how	strong	each	of	those	features	are	across	the	different	photos.	

2) Reconstruct a face.  How many EOFs do you need to reconstruct a face from the database?  Does it depend on the face that it used?
    - For	a man (face #35)	I	need	about	30	EOFs	to	recreate it to an accuracy of 95% correlation,	whereas	for	a	woman	(face	#340)	I	need	about 100 EOFs. It highly depends on the face used, because some faces have more features that require less variance to recreate it, such as for a man. With less data for a woman's facial features, you require more EOFs to recreate it.

3) Food for thought: The database contains 75% white men.  How do you think this database sampling limitation impacts the utility of the database for subjects who are not white men?  What are some parallels that you might draw when analyzing atmospheric and oceanic sciences datasets?  Hint: Think about the limitations of extrapolation beyond the domain where you have data.
    - Not	including	woman	or	people	of	color	means	that	features	which	explain	variance in	white	men	are	emphasized	in	the	EOFs	over	features	which	explain	variance	in	all	 people!
    - Many arctic models rely on year-long observational data that can capture full seasonal cycles. However, it is hard to that and so far, very few data has been collected during a full seasonal cycle of the sea-ice in the arctic for example. This can result in some bias of the data. Most observations are done during the summer when the weather is much more favorable, and thus temporal extrapolation would miss a lot of crucial information.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
%matplotlib inline

In [ ]:
#load faces. Adapted from the AT&T face database
att_faces = np.load('att_faces.npy')
height, width, n_faces = att_faces.shape
print((height, width, n_faces))

In [ ]:
# f, ax = plt.subplots(40, 1, figsize=(5,75))
# for i,j in enumerate(range(0,400,10)):
#     ax[i].imshow(att_faces[:, :, j], cmap='binary_r')
#     ax[i].set_title('Face idx#: ' + str(j))
#     ax[i].axis('off')

In [ ]:
#print a few faces from the database - Look at your data!!
n = 2
fig, axs = plt.subplots(1, n)

random_face_inds = np.random.choice(np.arange(n_faces), n, replace=False)
for i, ax in enumerate(axs):
    ax.imshow(att_faces[:, :, random_face_inds[i]], cmap = 'gray')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

fig.tight_layout()

In [ ]:
#print the average face
average_face = np.mean(att_faces, axis = 2);
plt.imshow(average_face, cmap = 'gray');

In [ ]:
#Construct the data matrix. Samples (different faces) along rows and 
#variables (individual pixels) along columns
print(att_faces.shape)
data_matrix = np.reshape(att_faces, (height*width, n_faces)).T
print(data_matrix.shape)

#Construct anomaly matrix by removing the average face, i.e., 
#by removing the column means from each column of the data_matrix
anomaly_matrix = data_matrix - np.mean(data_matrix, axis=0)
#anomaly_matrix = anomaly_matrix/np.std(anomaly_matrix, axis=0)
[eofs, s, v] = np.linalg.svd(anomaly_matrix.T, full_matrices=False)
print(eofs.shape)

eigenvals = np.square(s)
variance_explained = eigenvals/np.sum(eigenvals)

In [ ]:
#Plot the top 15 eofs (a.k.a eigenfaces)

fig, axs = plt.subplots(3, 5)

for i, ax in enumerate(np.reshape(axs, 15)):
    eigenface = np.reshape(eofs[:, i], [height, width])
    ax.imshow(eigenface, cmap='gray')
    ax.set_title('eigenface' + str(i+1))
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

fig.tight_layout()

In [ ]:
#Plot variance explained by each eigenface
plt.plot(variance_explained[0:100]*100, marker='o')
plt.xlim(0,15)
plt.ylim(bottom=0)

In [ ]:
threshold = 75 #percent of variance retained

ret_var = 0
for i in range(len(variance_explained)):
    ret_var = ret_var + variance_explained[i]*100
    if ret_var > threshold:
        print(f'{i+1} EOFs Needed')
        break

In [ ]:
#project the faces onto the eigenfaces
weights = np.dot(anomaly_matrix, eofs)
print(weights.shape)

#choose a face to reconstruct
face_num = 340 #0 to 389
original_face = att_faces[:, :, face_num]

#choose number of weights to reconstruct face
weights_num = 97
face_weights = weights[face_num, 0:weights_num]
print(face_weights.shape)
reconstructed_face = np.dot(eofs[:, 0:weights_num], face_weights)

#At this point, the reconstructed face is an anomaly face with the average face removed. 
#Add back the average face to get the original face

reconstructed_face = np.reshape(reconstructed_face, (height, width)) + average_face
print(reconstructed_face.shape)
fig, axs = plt.subplots(1, 2)
axs[0].imshow(original_face, cmap='gray');
axs[1].imshow(reconstructed_face, cmap='gray');

In [ ]:
%%time

threshold = 0.95 #correlation retained

#choose a face to reconstruct
face_num = 340 #0 to 389
original_face = att_faces[:, :, face_num]
og_face = xr.DataArray(original_face)

for i in range(len(variance_explained)):
    weights_num = i+1
    face_weights = weights[face_num, 0:weights_num]
    reconstructed_face = np.dot(eofs[:, 0:weights_num], face_weights)
    reconstructed_face = np.reshape(reconstructed_face, (height, width)) + average_face
    rc_face = xr.DataArray(reconstructed_face)
    corr_face = xr.corr(og_face, rc_face).values
    if corr_face > threshold:
        print(f'{weights_num} EOFs Needed')
        break